# December 12, 2023

https://adventofcode.com/2023/day/12

In [1]:
import re

In [2]:
def parse_input( lines ):
    data = []
    groups = []
    for line in lines:
        d, g = line.split()
        g = [int(x) for x in g.split(",")]
        data.append(d)
        groups.append(g)

    return {'data': data, 'groups':groups}


In [3]:
test_str = f'''???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1'''
test = parse_input( test_str.split("\n") )

In [4]:
fn = "data/12.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz = parse_input( [x.strip() for x in text] )

### Part 1

In [74]:
def count_solutions( text, groups, verbose=False ):
    '''
    text: string for this row of the map
       groups: sizes of broken strings to search for
    '''
    if verbose:
        print("Remaining string:", text)
        print(groups)
    
    # BASE CASES
    if len(groups) == 0:
        if "#" in text:
            # no solutions, because we have a # unaccounted for
            return 0
        else:
            # one solution: all ? stand for .
            return 1

    # First trim out the initial .s
    m = re.search("^\.*", text)
    if m is not None:
        text = text[ m.span()[1]: ]      

    if len(text) == 0:
        # no solutions because we're out of text and have unmatched groups
        return 0
    
    # Now we can assume that the first char is ? or #
    # check for initial string of G ?s or #s
    pattern = f'''^[\?#]{{{groups[0]}}}(?!#)'''

    tot_solutions = 0
    while len(text) >= sum(groups) + len(groups) - 1:
        m = re.search( pattern, text )
        if m is not None:
            # found a match!
            # now solve the smaller problem with one less group
            tot_solutions += count_solutions( text[ groups[0]+1 :], groups[1:], verbose=verbose )
        
        # if first char is #, then we can't advance cursor and try again
        if text[0] == "#":
            break

        # otherwise, trim leading ? and try again
        text = text[1:]         

    return tot_solutions




In [34]:
test['data'][0], test['groups'][0]

('???.###', [1, 1, 3])

In [75]:
count_solutions( test['data'][0], test['groups'][0], verbose=True )

Remaining string: ???.###
[1, 1, 3]
Remaining string: ?.###
[1, 3]
Remaining string: ###
[3]
Remaining string: 
[]


1

In [107]:
def part1( puzz ):
    tot = 0
    i = 0
    for d,g in zip(puzz['data'], puzz['groups']):
        tot += count_solutions( d, g )
        print( f'''[{i}] {tot}''')
        i += 1
    return tot

In [53]:
count_solutions(test['data'][0], test['groups'][0])

1

In [76]:
for d,g in zip(test['data'], test['groups']):
    print("\n\n\n")
    print(d,g)
    print( count_solutions(d,g, verbose=True) )






???.### [1, 1, 3]
Remaining string: ???.###
[1, 1, 3]
Remaining string: ?.###
[1, 3]
Remaining string: ###
[3]
Remaining string: 
[]
1




.??..??...?##. [1, 1, 3]
Remaining string: .??..??...?##.
[1, 1, 3]
Remaining string: ..??...?##.
[1, 3]
Remaining string: ...?##.
[3]
Remaining string: 
[]
Remaining string: ..?##.
[3]
Remaining string: 
[]
Remaining string: .??...?##.
[1, 3]
Remaining string: ...?##.
[3]
Remaining string: 
[]
Remaining string: ..?##.
[3]
Remaining string: 
[]
Remaining string: ...?##.
[1, 3]
Remaining string: ..?##.
[1, 3]
4




?#?#?#?#?#?#?#? [1, 3, 1, 6]
Remaining string: ?#?#?#?#?#?#?#?
[1, 3, 1, 6]
Remaining string: #?#?#?#?#?#?
[3, 1, 6]
Remaining string: #?#?#?#?
[1, 6]
Remaining string: #?#?#?
[6]
Remaining string: 
[]
1




????.#...#... [4, 1, 1]
Remaining string: ????.#...#...
[4, 1, 1]
Remaining string: #...#...
[1, 1]
Remaining string: ..#...
[1]
Remaining string: ..
[]
1




????.######..#####. [1, 6, 5]
Remaining string: ????.######..#####.
[1, 

In [108]:
part1(test)

[0] 1
[1] 5
[2] 6
[3] 7
[4] 11
[5] 21


21

In [109]:
# too high 8838
# too high 7615
# too high 7255
part1(puzz)

[0] 3
[1] 7
[2] 23
[3] 34
[4] 43
[5] 95
[6] 97
[7] 106
[8] 124
[9] 128
[10] 190
[11] 192
[12] 195
[13] 205
[14] 237
[15] 242
[16] 266
[17] 268
[18] 273
[19] 276
[20] 278
[21] 284
[22] 288
[23] 294
[24] 299
[25] 302
[26] 339
[27] 343
[28] 346
[29] 353
[30] 388
[31] 392
[32] 395
[33] 396
[34] 400
[35] 403
[36] 415
[37] 420
[38] 421
[39] 422
[40] 427
[41] 432
[42] 444
[43] 446
[44] 452
[45] 456
[46] 462
[47] 465
[48] 466
[49] 468
[50] 472
[51] 504
[52] 510
[53] 516
[54] 549
[55] 552
[56] 564
[57] 601
[58] 608
[59] 616
[60] 618
[61] 619
[62] 625
[63] 626
[64] 628
[65] 638
[66] 641
[67] 644
[68] 646
[69] 647
[70] 648
[71] 652
[72] 653
[73] 657
[74] 663
[75] 664
[76] 668
[77] 679
[78] 685
[79] 692
[80] 693
[81] 694
[82] 709
[83] 715
[84] 743
[85] 755
[86] 780
[87] 785
[88] 792
[89] 796
[90] 799
[91] 813
[92] 814
[93] 815
[94] 820
[95] 821
[96] 823
[97] 828
[98] 832
[99] 833
[100] 835
[101] 848
[102] 852
[103] 854
[104] 862
[105] 877
[106] 879
[107] 883
[108] 886
[109] 932
[110] 952
[111] 953

7173

### Part 2

In [112]:
def embiggen_puzzle( puzz ):
    big_puzz = {}

    return {
        'data': [ "?".join( [x]*5 ) for x in puzz['data'] ],
        'groups': [ x*5 for x in puzz['groups'] ]
    }

In [99]:
btest = embiggen_puzzle(test)

In [100]:
btest

{'data': ['???.###????.###????.###????.###????.###',
  '.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.',
  '?#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#?',
  '????.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...',
  '????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.',
  '?###??????????###??????????###??????????###??????????###????????'],
 'groups': [[1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3],
  [1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3],
  [1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6],
  [4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1],
  [1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5],
  [3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1]]}

In [102]:
part1(btest)

525152

In [113]:
bpuzz = embiggen_puzzle(puzz)

In [114]:
part1(bpuzz)

[0] 3888
[1] 32757


KeyboardInterrupt: 